In [1]:
import sqlite3

import numpy as np

# Creating connection to a .sqlite3 db. This will create the file when the script runs for the first time
conn = sqlite3.connect('db_big.sqlite3')
# Taking the cursor
c = conn.cursor()

In [12]:
c.execute('''select inspection.facility_zip,  violation.points, strftime("%Y-%m",inspection.activity_date) from inspection join  violation on
inspection.serial_number = violation.serial_number
where inspection.activity_date between date('2015-07-01') and date('2017-12-31')
''')

data = c.fetchall()

In [32]:
# Converting to database result to numpy array
data = np.asarray(data)

In [109]:
#deriving unique post codes in all data
all_post_codes = np.unique(data[:,0])

all_post_codes_with_violations = np.empty((3005,2), dtype=object)

In [111]:

for i in range(len(all_post_codes)):
    total_violations = np.sum(data[np.where(data[:,0] == all_post_codes[i])][:,1].astype(np.int32))
    
    na=np.array([all_post_codes[i], total_violations])
    
    all_post_codes_with_violations[i] = na
    
total_violations = all_post_codes_violations[:,1].astype(np.int)
sorted_violations = np.sort(total_violations)[-10:]

In [137]:
# finding postcodes associated with top violations

top_postcodes = []

for violation in sorted_violations:
    p = all_post_codes_with_violations[np.where(all_post_codes_with_violations[:,1]==str(violation))]
    top_postcodes.append(p[0][0])

In [138]:
top_postcodes

['90650',
 '91776',
 '90036',
 '90005',
 '90045',
 '90006',
 '90028',
 '90020',
 '90012',
 '91748']